### Convolutional Token Embedding
- ViT에서는 이미지를 고정된 크기의 패치로 분할해, 이를 flatten해 사용
- CvT에서는 이미지 혹은 이전 스테이지의 2D-shape token map에 대해 convolution을 적용한 뒤, 이를 flatten해 사용

In [6]:
import torch
import torch.nn as nn

# ViT의 패치 임베딩 레이어
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)  # [B, E, H, W]
        x = x.flatten(2)  # [B, E, N]
        x = x.transpose(1, 2)  # [B, N, E]
        return x

# CvT의 컨볼루션 토큰 임베딩 레이어
class ConvTokenEmbedding(nn.Module):
    def __init__(self, in_channels=3, out_channels=768, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

    def forward(self, x):
        x = self.conv(x)  # [B, E, H, W]
        x = x.flatten(2)  # [B, E, N]
        x = x.transpose(1, 2)  # [B, N, E]
        return x

# 더미 이미지 데이터 생성
dummy_img = torch.randn(1, 3, 224, 224)  # [B, C, H, W]

# 모델 초기화
vit_patch_embedding = PatchEmbedding()
cvt_conv_embedding = ConvTokenEmbedding()

# 더미 데이터를 통과시키기
vit_output = vit_patch_embedding(dummy_img)
cvt_output = cvt_conv_embedding(dummy_img)

vit_output.shape, cvt_output.shape

(torch.Size([1, 196, 768]), torch.Size([1, 50176, 768]))

### Convolutional Projection
- ViT에서는 전달된 토큰에 대해 선형 레이어(Linear Layer)를 이용해 qkv를 투사(projection)하고,
- CvT에서는 전달된 토큰에 대해 컨볼루션(Convolution Layer)을 이용해 qkv를 투사(projection)함.
 - 정확히는 Linear Projection하기 전에 Convolution Layer를 한 번 더 통과하는 게 맞음

In [23]:
# ViT의 Q, K, V 투사 레이어
class LinearProjection(nn.Module):
    def __init__(self, dim, heads):
        super().__init__()
        self.heads = heads
        self.scale = (dim // heads) ** -0.5

        # Q, K, V에 대한 선형 투사
        self.qkv = nn.Linear(dim, dim * 3)

    def forward(self, x):
        b, n, _ = x.shape
        qkv = self.qkv(x).reshape(b, n, 3, self.heads, -1).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        return q, k, v

# CvT의 Convolutional Projection 레이어 (수정된 버전)
class ConvProjectionCorrected(nn.Module):
    def __init__(self, dim, heads, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.heads = heads
        self.scale = (dim // heads) ** -0.5

        # Q, K, V에 대한 컨볼루션 레이어
        self.qkv = nn.Conv2d(dim, dim * 3, kernel_size, stride, padding, groups=heads)
        self.reshape = nn.Unflatten(2, (heads, dim // heads))

    def forward(self, x):
        b, _, h, w = x.shape
        x = self.qkv(x).view(b, self.heads, 3, -1, h, w).permute(2, 0, 1, 3, 4, 5)
        q, k, v = x[0], x[1], x[2]
        return q, k, v

# 더미 데이터 초기화 및 모델 정의
dummy_img = torch.randn(1, 768, 28, 28)  # CvT의 더미 데이터
dummy_seq = torch.randn(1, 196, 768)  # ViT의 더미 데이터 (패치 임베딩 후)

linear_projection = LinearProjection(dim=768, heads=12)
conv_projection_corrected = ConvProjectionCorrected(dim=768, heads=12)

# 더미 데이터를 통과시키기
vit_q, vit_k, vit_v = linear_projection(dummy_seq)
cvt_q, cvt_k, cvt_v = conv_projection_corrected(dummy_img)

(vit_q.shape, vit_k.shape, vit_v.shape), (cvt_q.shape, cvt_k.shape, cvt_v.shape)



((torch.Size([1, 12, 196, 64]),
  torch.Size([1, 12, 196, 64]),
  torch.Size([1, 12, 196, 64])),
 (torch.Size([1, 12, 64, 28, 28]),
  torch.Size([1, 12, 64, 28, 28]),
  torch.Size([1, 12, 64, 28, 28])))

In [28]:
from einops import rearrange

# CvT의 멀티헤드 어텐션 레이어 수정
class CvTattention(nn.Module):
    def __init__(self, dim, num_heads, kernel_size=3, stride_kv=2, stride_q=1, padding_kv=1, padding_q=1):
        super().__init__()
        self.num_heads = num_heads
        self.dim = dim
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5

        # 컨볼루션 프로젝션
        self.conv_proj_q = nn.Conv2d(dim, dim, kernel_size=kernel_size, padding=padding_q, stride=stride_q, groups=dim)
        self.conv_proj_k = nn.Conv2d(dim, dim, kernel_size=kernel_size, padding=padding_kv, stride=stride_kv, groups=dim)
        self.conv_proj_v = nn.Conv2d(dim, dim, kernel_size=kernel_size, padding=padding_kv, stride=stride_kv, groups=dim)

    def forward(self, x):
        q = self.conv_proj_q(x)
        k = self.conv_proj_k(x)
        v = self.conv_proj_v(x)

        q = rearrange(q, 'b c h w -> b (h w) c')
        k = rearrange(k, 'b c h w -> b (h w) c')
        v = rearrange(v, 'b c h w -> b (h w) c')

        return q, k, v
    
# 더미 데이터 초기화 및 모델 정의
dummy_img_vit = torch.randn(1, 196, 768)  # ViT의 더미 데이터 (패치 임베딩 후)
dummy_img_cvt = torch.randn(1, 768, 14, 14)  # CvT의 더미 데이터 (컨볼루션 후)

# CvT의 멀티헤드 어텐션 모델 정의 및 더미 데이터 통과
cvt_attention = CvTattention(dim=768, num_heads=12, kernel_size=3, stride_kv=2, stride_q=1)
cvt_q, cvt_k, cvt_v = cvt_attention(dummy_img_cvt)

# 각 모델의 출력 형태
(vit_q.shape, vit_k.shape, vit_v.shape), (cvt_q.shape, cvt_k.shape, cvt_v.shape)

((torch.Size([1, 12, 196, 64]),
  torch.Size([1, 12, 196, 64]),
  torch.Size([1, 12, 196, 64])),
 (torch.Size([1, 196, 768]),
  torch.Size([1, 49, 768]),
  torch.Size([1, 49, 768])))